# Web Coverage Service (WCS) Download Example
## Introduction
We'll demonstrate how to download a GeoTIFF data file from a public WCS service using Python 3.  
### WCS Data Service
For this demonstration we'll use Landfire (LF_1.4.0): https://www.landfire.gov/data_access.php
For Landfire LF_1.4.0 we see that the base URL is https://landfire.cr.usgs.gov/arcgis/services/Landfire/US_140/MapServer/WCSServer
## WCS Requests
There are three types of WCS requests:
- GetCapabilities
- DescribeCoverage
- GetCoverage

Generally, you first do a GetCapabilities request to obtain high-level information about what data you can ask for from the WCS server.  
You then perform a DescribeCoverage request to get information specific to the coverage you want to get data from.  
Finally, you perform a GetCoverage to obtain the data itself.
### GetCapabilities
Let's perform a GetCapabilities request on the Landfire WCS server to see what the service can do.

In [7]:
import requests

# base WCS server URL
wcs_base_url = "https://landfire.cr.usgs.gov/arcgis/services/Landfire/US_140/MapServer/WCSServer"

# add on to the base WCS server URL to define GetCapabilities URL
wcs_get_capabilities_url = wcs_base_url + "?REQUEST=GetCapabilities&SERVICE=WCS"

# perform an HTTP GET request with the GetCapabilities URL
wcs_get_capabilities_response = requests.get(wcs_get_capabilities_url)

# show the resulting body of the GetCapabilities request
print("GetCapabilities Response:")
print(wcs_get_capabilities_response.text)

GetCapabilities Response:
<?xml version="1.0" encoding="UTF-8"?>
<Capabilities xmlns="http://www.opengis.net/wcs/1.1"
 xmlns:ows="http://www.opengis.net/ows/1.1"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 xmlns:xlink="http://www.w3.org/1999/xlink"
 xsi:schemaLocation="http://www.opengis.net/wcs/1.1 http://schemas.opengis.net/wcs/1.1/wcsGetCapabilities.xsd http://www.opengis.net/ows/1.1/ http://schemas.opengis.net/ows/1.1.0/owsAll.xsd" version="1.1.2">
  <ows:ServiceIdentification>
    <ows:Title>US_140</ows:Title>
  <ows:ServiceType>WCS</ows:ServiceType>
  <ows:ServiceTypeVersion>1.0.0</ows:ServiceTypeVersion>
  <ows:ServiceTypeVersion>1.1.0</ows:ServiceTypeVersion>
  <ows:ServiceTypeVersion>1.1.1</ows:ServiceTypeVersion>
  <ows:ServiceTypeVersion>1.1.2</ows:ServiceTypeVersion>
  <ows:Fees>NONE</ows:Fees>
  <ows:AccessConstraints>NONE</ows:AccessConstraints>
  </ows:ServiceIdentification>
  <ows:ServiceProvider>
    <ows:ProviderName></ows:ProviderName>
    <ows:ServiceCon

#### GetCapabilities results
The GetCapabilities request returns a lot of information for us.  

We can see that the WCS server supports WCS version `1.0.0`, `1.1.0`, `1.1.1`, and `1.1.2` .  
For the purposes of this guide, we're going to stick to WCS 1.0.0.

Within the contents section, we can see all of the available coverages.  A coverage is just another word for a data layer.
For this guide, let's pick an arbitrary data layer: `US_VDIST2014`

### DescribeCoverage

We'll use the results we got from GetCoverage to perform a DescribeCoverage request to get more specific information about how to ultimately perform the GetCoverage Operation.  We'll use WCS version `1.0.0` with coverage `US_VDIST2014` to perform the request.

In [4]:
import requests

# base WCS server URL
wcs_base_url = "https://landfire.cr.usgs.gov/arcgis/services/Landfire/US_140/MapServer/WCSServer"

# add on to the base WCS server URL to define DescribeCoverage URL
wcs_describe_coverage_url = wcs_base_url + "?SERVICE=WCS&VERSION=1.0.0&REQUEST=DescribeCoverage&COVERAGE=US_VDIST2014"

# perform an HTTP GET request with the DescribeCoverage URL
wcs_describe_coverage_response = requests.get(wcs_describe_coverage_url)

# show the resulting body of the DescribeCoverage request
print("DescribeCoverage Response:")
print(wcs_describe_coverage_response.text)

DescribeCoverage Response:
<?xml version="1.0" encoding="UTF-8"?>
<CoverageDescription xmlns="http://www.opengis.net/wcs"
 xmlns:gml="http://www.opengis.net/gml"
 xmlns:xlink="http://www.w3.org/1999/xlink"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 xsi:schemaLocation="http://www.opengis.net/wcs http://schemas.opengeospatial.net/wcs/1.0.0/describeCoverage.xsd" version="1.0.0">
<CoverageOffering>
    <description>tempname</description>
    <name>US_VDIST2014</name>
    <label>US_VDIST2014</label>
    <lonLatEnvelope srsName="urn:ogc:def:crs:OGC:1.3:CRS84">
      <gml:pos dimension="2">-127.98775263969655 22.765446426860603</gml:pos>
      <gml:pos dimension="2">-65.254445466369276 51.649681015029245</gml:pos>
    </lonLatEnvelope>
    <domainSet>
      <spatialDomain>
        <gml:Envelope srsName="EPSG:102039">
          <gml:pos dimension="2">-2362425 258915</gml:pos>
          <gml:pos dimension="2">2263815 3177435</gml:pos>
        </gml:Envelope>
        <gml:RectifiedG

#### DescribeCoverage results
From the DescribeCoverage response, we can see information about the available bounding box for the layer, the Coordinate Reference System (CRS), the download data types (e.g. GeoTiff), and more.  We'll use this information to help formulate a successful WCS GetCoverage request to download some actual data.

### GetCoverage
With the results from DescribeCoverage, we now know everything we need to know in order to download some data using a GetCoverage request.
One unfortunate aspect of WCS version 1.0.0 is that you must provide the resolution or number of pixels of your downloaded output file; you cannot omit this and simply ask for the download file to be in the native resolution of the source data.  This means that you may need to perform some additional math to calculate the native resolution on your own before downloading the file.  For the purposes of this tutorial, we'll simply download a 500 by 100 pixel image of the continental United States.  

In [6]:
import requests

# base WCS server URL
wcs_base_url = "https://landfire.cr.usgs.gov/arcgis/services/Landfire/US_140/MapServer/WCSServer"

# add on to the base WCS server URL to define GetCoverage URL
wcs_get_coverage_url = wcs_base_url + "?SERVICE=WCS&VERSION=1.0.0&REQUEST=GetCoverage&FORMAT=GeoTIFF&COVERAGE=US_VDIST2014&BBOX=-127.98775263969655,22.765446426860603,-65.254445466369276,51.649681015029245&CRS=EPSG:4326&WIDTH=500&HEIGHT=100"

# perform an HTTP GET request with the DescribeCoverage URL
wcs_get_coverage_response = requests.get(wcs_get_coverage_url)

# download the resulting response image to disk
if wcs_get_coverage_response.status_code == 200:
    with open("wcs-example.tif", 'wb') as f:
        f.write(wcs_get_coverage_response.content)

#### GetCoverage results
At this point you should have a downloaded GeoTiff file called `wcs-example.tif`.  Once you use the GetCapabilities and DescribeCoverage requests to dial in the types of GetCoverage requests you want to perform, you can simply tweak your GetCoverage requests to perform these types of requests in an automated fashion on your code.